#Understanding per-example feature importance

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set_context('notebook')
sns.set_style('darkgrid')
import sklearn as skl
import sklearn.metrics
import sklearn.learning_curve
import sklearn.ensemble
import sklearn.dummy
import sklearn.grid_search
import sklearn.feature_extraction
import sklearn.feature_extraction.text
import numpy as np
import pickle

import sys
sys.path = sys.path + ['..']
import get_airbnb_data
import db
import learning
import imp
imp.reload(learning)
from learning import *

In [2]:
clf = pickle.load(open('../pipe2.pkl', 'rb'))

In [3]:
engine = db.create_root_engine()
rawtable = pd.io.sql.read_sql_table('listings', engine, index_col='id')
Xtr, Xte, ytr, yte = get_training_test_set(rawtable, make_features=make_features5, 
                                               categorize_rating=categorize_rating5)

In [4]:
single_example = Xtr.iloc[0, :]
comparison_examples = Xtr.iloc[1:101, :].reset_index(drop=True)  # Will need to select these randomly
#comparison_examples

In [5]:
single_proba = clf.predict_proba(single_example)
class_ = np.argmax(single_proba)

In [18]:
np.repeat(0, 100)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [29]:
Xtr.columns.get_loc('lat')

62

In [44]:
Xtr.columns[52]

'beds'

In [42]:
column = 52

In [45]:
single_example

amenity0                          1
amenity1                          0
amenity2                          1
amenity3                          0
amenity4                          0
amenity5                          0
amenity6                          0
amenity7                          0
amenity8                          0
amenity9                          0
amenity10                         0
amenity11                         0
amenity12                         0
amenity13                         0
amenity14                         0
amenity15                         1
amenity16                         0
amenity17                         0
amenity18                         0
amenity19                         0
amenity20                         0
amenity21                         1
amenity22                         1
amenity23                         0
amenity24                         0
amenity25                         0
amenity26                         0
amenity27                   

In [6]:
importances = []
for column in range(97):
    # Need to reset index because we are combining data with different id so don't want to align by index
    scrambled_examples = pd.DataFrame(single_example, copy=True).T.iloc[np.repeat(0, 100), :].reset_index(drop=True)
    scrambled_examples.iloc[:, column] = comparison_examples.iloc[:, column]
# Either big spread or mean offset of scrambled predictions - normal prediction means feature was important
#   so take RMS
    importances.append(np.mean((clf.predict_proba(scrambled_examples)[:, class_] - single_proba[0, class_]) ** 2))

In [7]:
for i, importance in enumerate(importances):
    print(Xtr.columns[i], importance)

amenity0 1.10987136149e-06
amenity1 5.10821643905e-05
amenity2 0.0
amenity3 0.0
amenity4 0.000459181475414
amenity5 0.0
amenity6 0.0
amenity7 0.0
amenity8 0.000252177160473
amenity9 5.09399247278e-05
amenity10 4.3646047005e-05
amenity11 4.98416906454e-06
amenity12 0.0
amenity13 0.0
amenity14 0.0
amenity15 1.49525071936e-06
amenity16 0.000327232256871
amenity17 0.000222851496352
amenity18 0.0
amenity19 1.1512926355e-06
amenity20 4.02952422425e-06
amenity21 0.000163616128436
amenity22 4.66990248223e-06
amenity23 0.0
amenity24 0.0
amenity25 0.0
amenity26 0.0
amenity27 0.0
amenity28 0.0
amenity29 0.0
amenity30 0.0
amenity31 0.0
amenity32 0.0
amenity33 0.0
amenity34 0.0
amenity35 0.0
amenity36 0.0
amenity37 0.0
amenity38 0.0
amenity39 0.0
amenity40 0.0
amenity41 0.0
amenity42 0.0
amenity43 0.0
amenity44 0.0
amenity45 0.0
amenity46 0.0
amenity47 0.0
amenity48 0.0
amenity49 0.0
bathrooms 0.000321032407587
bedrooms 0.000143634946165
beds 0.000258744714476
instantBookable 7.27182793048e-05
isCa

In [8]:
np.argmax(importances)

63

In [12]:
np.argsort(importances)[::-1]

array([63, 73, 56, 88, 96, 85, 95, 69, 93, 94, 91, 67, 61,  4, 83, 68, 70,
       16, 50, 60, 59, 71, 64, 52,  8, 65, 66, 17, 62, 57, 21, 51, 55, 58,
       75, 53, 80,  1,  9, 10, 72, 76, 82, 92, 11, 22, 20, 15, 19,  0, 89,
       86, 46, 12, 54, 74, 18, 77, 78, 79, 14, 13, 84, 81, 47, 87,  7,  6,
        5,  3,  2, 90, 49, 23, 45, 35, 44, 43, 42, 41, 40, 39, 38, 37, 36,
       34, 24, 33, 32, 31, 30, 29, 28, 27, 26, 25, 48])